In [7]:
import numpy as np
import torch

from GOLCNNInspector import GOLCNNInspector
from MinimalSolution import MinNet
from GOLCNN import OPNet

In [8]:
# Creates a 32x32 field to play Life on and populates it with a glider pattern
glider = np.zeros((32,32))
glider[1,3]=1
glider[2,3]=1
glider[3,3]=1
glider[3,2]=1
glider[2,1]=1

In [9]:
# Creates a 32x32 field to play Life on and populates it with a pulsar pattern
pulsar = np.zeros((32,32))
pulsar[7,6] = pulsar[8,6] = pulsar[9,6] = pulsar[13,6] = pulsar[14,6] = pulsar[15,6] = 1
pulsar[7,11] = pulsar[8,11] = pulsar[9,11] = pulsar[13,11] = pulsar[14,11] = pulsar[15,11] = 1
pulsar[7,13] = pulsar[8,13] = pulsar[9,13] = pulsar[13,13] = pulsar[14,13] = pulsar[15,13] = 1
pulsar[7,18] = pulsar[8,18] = pulsar[9,18] = pulsar[13,18] = pulsar[14,18] = pulsar[15,18] = 1
pulsar[5,8] = pulsar[5,9] = pulsar[5,10] = pulsar[5,14] = pulsar[5,15] = pulsar[5,16] = 1
pulsar[10,8] = pulsar[10,9] = pulsar[10,10] = pulsar[10,14] = pulsar[10,15] = pulsar[10,16] = 1
pulsar[12,8] = pulsar[12,9] = pulsar[12,10] = pulsar[12,14] = pulsar[12,15] = pulsar[12,16] = 1
pulsar[17,8] = pulsar[17,9] = pulsar[17,10] = pulsar[17,14] = pulsar[17,15] = pulsar[17,16] = 1

In [16]:
# Creates a 32x32 field to play Life on and populates it with a random pattern
random = np.random.randint(0,2,(32,32))

In [17]:
mynet = MinNet(2)
testnet = torch.load(f'./models/op_m16_n2_model1.pt')
testnet.to('cpu')
print('successfully loaded models')

successfully loaded models


In [18]:
inspector = GOLCNNInspector(mynet, pulsar)

MultiCanvas(height=512, width=512)

GridspecLayout(children=(Button(description='Reset', layout=Layout(grid_area='widget001'), style=ButtonStyle()…

Output()

In [13]:
inspector2 = GOLCNNInspector(testnet, pulsar)

MultiCanvas(height=512, width=512)

GridspecLayout(children=(Button(description='Reset', layout=Layout(grid_area='widget001'), style=ButtonStyle()…

Output()